# Generación de bdd vectorial con pinecone

En este notebook se presenta la construcción de la base de datos vectorial utilizada como la principal fuente de información para el proyecto. La base de datos contiene generalidades de la mina, protocolos de emergencia e imágenes que representan planos.


Configuración del apikey para pinecone

In [1]:
import os
from dotenv import load_dotenv

# Caragar archivo .env con el api_key
load_dotenv()

# Obtener la clave API 
api_key = os.getenv("api_key_pinecone")
if not api_key:
    raise ValueError("The 'api_key' environment variable is not defined in the .env file.")

# Inicializa Pinecone con tu API key
from pinecone import Pinecone, ServerlessSpec
pc = Pinecone(api_key=api_key)


Definición de la base de datos vectorial

In [2]:
index_name = "info-mina"

pc.create_index(
    name=index_name,
    dimension=1536, # Replace with your model dimensions
    metric="cosine", # Replace with your model metric
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"
    ) 

)

PineconeApiException: (409)
Reason: Conflict
HTTP response headers: HTTPHeaderDict({'content-type': 'text/plain; charset=utf-8', 'access-control-allow-origin': '*', 'vary': 'origin,access-control-request-method,access-control-request-headers', 'access-control-expose-headers': '*', 'x-pinecone-api-version': '2024-07', 'X-Cloud-Trace-Context': '1e136bcd2a194205b232784dff0056fd', 'Date': 'Thu, 09 Jan 2025 14:06:29 GMT', 'Server': 'Google Frontend', 'Content-Length': '85', 'Via': '1.1 google', 'Alt-Svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000'})
HTTP response body: {"error":{"code":"ALREADY_EXISTS","message":"Resource  already exists"},"status":409}


Lectura de información sobre la mina en archivo .csv (protocolos de seguridad), como formato diccionario.

In [256]:
csv_file="data\info_general.csv"
import pandas as pd

# Función para leer el archivo CSV y convertir las filas en diccionarios
def generar_diccionarios(csv_file):
    # Leemos el archivo CSV con pandas
    df = pd.read_csv(csv_file)
    
    # Convertimos cada fila en un diccionario
    diccionarios = df.to_dict(orient='records')
    
    return diccionarios
data=generar_diccionarios(csv_file)


Lectura de las imágenes (planos de la mina), y conversión a texto en base64

In [257]:
import base64
from PIL import Image
import io

# Función para codificar una imagen en base64
def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')

# Rutas de las 6 imágenes
image_paths = [
    "data\plano_general.jpg",
    "data\plano_1750.jpg",
    "data\plano_1810.jpg",
    "data\plano_1850.jpg",
    "data\plano_1900.jpg",
    "data\plano_1950.jpg"
]

# Codificamos cada imagen en base64 y almacenamos los resultados en una lista
base64_images = [encode_image(image_path) for image_path in image_paths]

Interpretación de las imágenes mediante GPT4o, y adición de filas a la lista de diccionarios data, que conformará la base de datos vectorial.

In [258]:
from openai import OpenAI
import os
from dotenv import load_dotenv

# Caragar archivo .env con el api_key
load_dotenv()

# Obtener la clave API 
api_key = os.getenv("api_key_openai")
if not api_key:
    raise ValueError("The 'api_key' environment variable is not defined in the .env file.")

# Configura tu clave API de OpenAI aquí
client=OpenAI(api_key=api_key)

# Lista para almacenar las respuestas de las imágenes
resp_images = []

# Preparamos el mensaje para GPT-4
for i, base64_image in enumerate(base64_images):
    print(i)
    print(base64_image)
    response = client.chat.completions.create(
        model="gpt-4o", 
        messages=[
            {
                "role": "system",
                "content": [
                    {
                        "type": "text",
                        "text": "Eres un experto en minería analizando planos."
                    }
                ]
            },
            {
                "role": "user",
                "content": [
                    {
                        "type": "image_url",
                        "image_url": {
                        "url": f"data:image/jpeg;base64,{base64_image}"}
                    },
                    {
                        "type": "text",
                        "text": "Proporciona el nombre del nivel y una descripción detallada y técnica de la imagen que te mostraré. No tienen escala, pero considera que la sección promedio de las obras es de 4.5 metros X 4.5 metros. Si tienes datos suficientes, determina método de minado."
                    }
                ]
            }
        ],
        response_format={  # Especificamos el formato de respuesta como texto
            "type": "text"
        },
        temperature=1,
        max_completion_tokens=2048,
        top_p=1,
        frequency_penalty=0,
        presence_penalty=0
    )

    # Almacenamos la descripción generada para cada imagen en la lista resp_images
    resp_images.append(response.choices[0].message.content)

import re

# Lista de rutas de imágenes
image_paths = [
    "data\\plano_general.jpg",
    "data\\plano_1750.jpg",
    "data\\plano_1810.jpg",
    "data\\plano_1850.jpg",
    "data\\plano_1900.jpg",
    "data\\plano_1950.jpg"
]

# Extraemos los nombres de las imágenes, excluyendo "data\\" y la extensión ".jpg"
imagenes = [path.split("\\")[-1].replace(".jpg", "") for path in image_paths]

# Aplicamos la transformación para eliminar el guion bajo y convertir a minúsculas
imagenes = [re.sub(r'(\_)', ' ', imagen.lower()).strip() for imagen in imagenes]

# añadir descripciones a los diccionarios en la columna "texto"
for i, imagen in enumerate(imagenes):
        # Creamos el diccionario para cada imagen
        nuevo_diccionario = {
            'id': imagen,
            'texto':  resp_images[i],  # Unimos el contenido de la lista en un solo string
            'categoria': 'plano',
            'fecha_creacion': '10/01/2025',
            'autor': 'departamento de planeacion, seguridad, operación, almacen, ti.'
        }
        
        # Agregamos el nuevo diccionario a la lista
        data.append(nuevo_diccionario)


0
...
1
...
2
...
3
...
4
...
5
...


Vectorización de la información, y adición de la columna embbadings a los diccionarios que conformarán la base de datos.

In [260]:
from openai import OpenAI
import os
from dotenv import load_dotenv
import json

# Caragar archivo .env con el api_key
load_dotenv()

# Obtener la clave API 
api_key = os.getenv("api_key_openai")
if not api_key:
    raise ValueError("The 'api_key' environment variable is not defined in the .env file.")

# Configura tu clave API de OpenAI aquí
client=OpenAI(api_key=api_key)

# Itera sobre cada elemento de la lista 'data' utilizando un índice 'i'
for i in range(len(data)):
    print(i)
    # Llama al método 'create' de 'client.embeddings' para obtener un embedding para el elemento 'data[i]'
    response = client.embeddings.create(
        # Convierte el elemento 'data[i]' a una cadena JSON, asegurándose de que los caracteres no ASCII se mantengan tal cual
        input=json.dumps(data[i], ensure_ascii=False),
        # Especifica el modelo que se utilizará para generar el embedding
        model="text-embedding-3-small"
    )

    # Almacena el embedding generado por el modelo en la clave "embeddings" del elemento correspondiente en 'data'
    # El embedding se extrae de 'response.data[0].embedding'
    data[i]["embeddings"] = response.data[0].embedding

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30


Reducción de resolución de planos para ajustarse a limite en versión de pinecone utilizada

In [284]:
import base64
from PIL import Image
import io
import os

# Función para abrir, comprimir y convertir una imagen en base64
def encode_image(image_path):
    # Abrir y comprimir la imagen
    img = Image.open(image_path)  # Abrir la imagen desde la ruta especificada
    
    # Convertir a RGB si es necesario (por si está en RGBA u otro formato)
    if img.mode != 'RGB':
        img = img.convert('RGB')

    # Redimensionar la imagen para reducir su tamaño
    max_size = (800, 800)  # Ajusta estos valores según necesites
    img.thumbnail(max_size, Image.Resampling.LANCZOS)

    # Comprimir la imagen con alta compresión
    buffer = io.BytesIO()
    img.save(buffer, format='JPEG', optimize=True, quality=20)  # Calidad reducida para mayor compresión
    buffer.seek(0)

    # Convertir a base64
    img_str = base64.b64encode(buffer.getvalue()).decode('utf-8')

    # Calcular y mostrar el tamaño en KB
    size_kb = len(img_str) / 1024
    print(f'Tamaño de la imagen en base64: {size_kb:.2f} KB')

    if size_kb > 30:
        print("¡Advertencia! La imagen sigue siendo mayor a 30KB. Considera reducir más la resolución o la calidad.")

    return img_str

# Lista para almacenar las imágenes codificadas en base64
base64_images_with_paths = []
# Ciclo for para procesar todas las imágenes
for image,imagen in zip(image_paths, imagenes):
    if os.path.exists(image):  # Verificar que el archivo existe
        print(f"Procesando {image}...")
        base64_image = encode_image(image)  # Codificar la imagen en base64
        base64_images_with_paths.append({
            "path": imagen,      # Guardar la ruta o nombre de la imagen
            "base64": base64_image   # Guardar la imagen codificada
        })
    else:
        print(f"Advertencia: El archivo {image} no existe.")


Procesando data\plano_general.jpg...
Tamaño de la imagen en base64: 25.55 KB
Procesando data\plano_1750.jpg...
Tamaño de la imagen en base64: 18.18 KB
Procesando data\plano_1810.jpg...
Tamaño de la imagen en base64: 19.40 KB
Procesando data\plano_1850.jpg...
Tamaño de la imagen en base64: 20.16 KB
Procesando data\plano_1900.jpg...
Tamaño de la imagen en base64: 20.71 KB
Procesando data\plano_1950.jpg...
Tamaño de la imagen en base64: 21.70 KB


Adicción de la columna imagen_b64 a los diccionarios. Las filas que no tienen una imagen asociada se rellenan con n/a

In [297]:
# Crear un diccionario para mapear 'path' con 'base64' en base64_images_with_paths
base64_dict = {item['path']: item['base64'] for item in base64_images_with_paths}

# Iterar sobre la lista data
for item in data:
    # Si el 'id' del item está en la lista imagenes
    if item['id'] in imagenes:
        # Obtener el valor de base64 usando el 'id' (asumido que el id es el 'path' en base64_images_with_paths)
        if item['id'] in base64_dict:
            item['imagen_b64'] = base64_dict[item['id']]  # Asignar el valor de base64
        else:
            print(f"Advertencia: No se encontró base64 para el id: {item['id']}")
    else:
        print(f"El id {item['id']} no está en la lista de imagenes.")

# Mostrar el resultado final de data
for item in data:
    print(item)


El id nivel_1900 no está en la lista de imagenes.
El id sensor_co_1900 no está en la lista de imagenes.
El id protocolo_incendio_1900 no está en la lista de imagenes.
El id info_general no está en la lista de imagenes.
El id nivel_1750 no está en la lista de imagenes.
El id sensor_co_1750 no está en la lista de imagenes.
El id protocolo_incendio_1750 no está en la lista de imagenes.
El id v1 no está en la lista de imagenes.
El id v2 no está en la lista de imagenes.
El id v3 no está en la lista de imagenes.
El id refugio_minero no está en la lista de imagenes.
El id autorescatador no está en la lista de imagenes.
El id taller_1900 no está en la lista de imagenes.
El id incendios_mina no está en la lista de imagenes.
El id nivel_1810 no está en la lista de imagenes.
El id sensor_co_1810 no está en la lista de imagenes.
El id protocolo_incendio_1810 no está en la lista de imagenes.
El id nivel_1850 no está en la lista de imagenes.
El id sensor_co_1850 no está en la lista de imagenes.
El i

A continuación, la información completa sobre la mina

In [317]:
df = pd.DataFrame(data)
df.tail(3)

,id,texto,categoria,fecha_creacion,autor,embeddings,imagen_b64
28,plano 1850,"El nivel indicado en la imagen es ""N-1850"". La...",plano,10/01/2025,"departamento de planeacion, seguridad, operaci...","[-0.04995483160018921, 0.052224233746528625, 0...",/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDACgcHiMeGSgjIS...
29,plano 1900,La imagen muestra el **nivel N-1900** de una m...,plano,10/01/2025,"departamento de planeacion, seguridad, operaci...","[-0.03590445965528488, 0.05903327465057373, 0....",/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDACgcHiMeGSgjIS...
30,plano 1950,### Nombre del Nivel\n- N-1950\n\n### Descripc...,plano,10/01/2025,"departamento de planeacion, seguridad, operaci...","[-0.029561417177319527, 0.044154662638902664, ...",/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDACgcHiMeGSgjIS...


In [255]:
#Este código se utilizó para eliminar todas las filas de la base vectorial durante el proceso de edición.
"""
# Conectar al índice usando pc
index = pc.Index("info-mina")

# Borrar todos los datos del índice
index.delete(delete_all=True, namespace="ns1")
"""


{}

Los datos anteriores se acomodan en tres columnas, llamadas id, values y metada, para ser interpretadas con pinecone.

In [306]:
import time
# Wait for the index to be ready
while not pc.describe_index(index_name).status['ready']:
    time.sleep(1)

index = pc.Index(index_name)

vectors = []
for d in data:
    vectors.append({
        "id": d['id'],
        "values": d['embeddings'],
        "metadata": {
            'texto': d['texto'],
            'categoria': d['categoria'],
            'fecha_creacion': d['fecha_creacion'],
            'autor': d['autor'],
            'imagen_b64': d.get('imagen_b64', '')  # Si 'imagen_b64' es nulo, se pone una cadena vacía
        }
    })

index.upsert(
    vectors=vectors,
    namespace="ns1"
)


{'upserted_count': 31}

Descripción de la base de datos vectorial

In [309]:
print(index.describe_index_stats())

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'ns1': {'vector_count': 31}},
 'total_vector_count': 31}


Generación de una query vectorizada mediante GTP

In [223]:
query = "hay un incendio en 1900, qué hago"

# Llama al método 'create' de 'client.embeddings' para obtener un embedding para el elemento 'data[i]'
response = client.embeddings.create(
   
    input=query,
    # Especifica el modelo que se utilizará para generar el embedding
    model="text-embedding-3-small"
    )

In [315]:
query ="plano general" #"hay un incendio en 1810, qué hago" +"extrae el plano, imagen 64"

# Llama al método 'create' de 'client.embeddings' para obtener un embedding para el elemento 'data[i]'
response_imagen = client.embeddings.create(
   
    input=query,
    # Especifica el modelo que se utilizará para generar el embedding
    model="text-embedding-3-small"
    )

Búsqueda de la query en la base de datos, considerando los 3 vectores más cercanos con distania coseno

In [225]:
results = index.query(
    namespace="ns1",
    vector=response.data[0].embedding,
    top_k=3,
    include_values=False,
    include_metadata=True
)

print(results)

{'matches': [{'id': 'protocolo_incendio_1900',
              'metadata': {'autor': 'departamento de planeacion, seguridad, '
                                    'operación, almacen, ti.',
                           'categoria': 'plan de emergencia',
                           'fecha_creacion': '09/01/2025',
                           'imagen_b64': '',
                           'texto': '"En caso de incendio en 1900 taller, si '
                                    'es un conato, usar extintores. Si el '
                                    'fuego se declara, evacuar inmediatamente. '
                                    'Cerrar puertas cortafuego del taller, '
                                    'aumentar al 95% V1, continuar evacuando '
                                    'mina. Si V1 no funciona, cerrar puertas '
                                    'corta fuego, aumentar a 95 % V2, apagar '
                                    'todos los ventiladores auxiliares en '
                    

In [316]:
results_imagen = index.query(
    namespace="ns1",
    vector=response_imagen.data[0].embedding,
    top_k=3,
    include_values=False,
    include_metadata=True,
    filter={ "imagen_b64": {"$ne": "n/a"}}  # Filtrar solo los que tienen valores en 'imagen_b64' 
)

print(results_imagen)

{'matches': [{'id': 'plano 1850',
              'metadata': {'autor': 'departamento de planeacion, seguridad, '
                                    'operación, almacen, ti.',
                           'categoria': 'plano',
                           'fecha_creacion': '10/01/2025',
                           'imagen_b64': '...',
                           'texto': 'El nivel indicado en la imagen es '
                                    '"N-1850". La imagen muestra un layout de '
                                    'túneles mineros con varios componentes '
                                    'clave:\n'
                                    '\n'
                                    '1. **Ventilación y Refugios:**\n'
                                    '   - **Ventiladores principales (Vx):** '
                                    'Hay al menos tres ventiladores '
                                    'principales (V2 y V3) ubicados en el '
                                    'sistema. Estos so

Código utilizado para decodificar una imagen en base64 y mostrarla.

In [313]:
import base64
from PIL import Image
from io import BytesIO# Extraer la cadena Base64 de la clave `imagen_b64`
imagen_b64 = results_imagen["matches"][0]["metadata"]["imagen_b64"]
# Decodificar la imagen
image_data = base64.b64decode(imagen_b64)

# Convertir a formato de imagen
image = Image.open(BytesIO(image_data))

# Mostrar la imagen
image.show()
